In [1]:
from scrapfly import ScrapflyClient, ScrapeConfig, ScrapflyError
 
scrapfly = ScrapflyClient(key="scp-live-01fc8534987042f4a2fe553c5cf6a2df", max_concurrency=2)
url = 'https://www.michaels.com/product/40oz-stainless-steel-insulated-tumbler-by-ashland-10751061'
listing_result = scrapfly.scrape(ScrapeConfig(url=url, render_js=True, country="US", asp=True, retry=False, rendering_wait=10000))
listing_html = listing_result.scrape_result['content']

In [2]:
from bs4 import BeautifulSoup
import re
import json
def extract_json_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tags = soup.find_all('script')

    json_data_list = []

    for script in script_tags:
        script_content = script.string
        if script_content:
            # Use a regular expression to find JSON-like content
            json_pattern = re.compile(r'({.*})', re.DOTALL)
            matches = json_pattern.findall(script_content)
            for match in matches:
                try:
                    json_data = json.loads(match)
                    json_data_list.append(json_data)
                except json.JSONDecodeError:
                    pass  # Ignore invalid JSON

    return json_data_list

json_data_list = extract_json_from_html(listing_html)

In [6]:
data = json_data_list[1]

In [9]:
sellername = None
site_url = None

for index in range(2):  # Assuming only the first two indices are relevant
    # Ensure the index is valid and that the 'name' is not None
    if index < len(json_data_list) and json_data_list[index].get('name') is not None:
        # Check if the current product name matches the desired one
            product_data = json_data_list[index]
            
            # Attempt to populate sellername and site_url if they exist
            if index + 1 < len(json_data_list):
                sellername = json_data_list[index + 1].get('@graph', [{}])[0].get('name')
                site_url = json_data_list[index + 1].get('@graph', [{}])[0].get('url')

            break


In [11]:
extracted_values = {
    'name': product_data.get('name', None),
    'url': product_data.get('url', None),
    'description': product_data.get('description', None),
    'category': product_data.get('category', None),
    'color': product_data.get('color', None),
    'brand_name': product_data.get('brand', {}).get('name', None),
    'image': product_data.get('image', None),
    'related_urls': product_data.get('isRelatedTo', {}).get('url', []),
    'currency': product_data.get('offers', {}).get('priceCurrency', None),
    'price': product_data.get('offers', {}).get('price', None),
    'price_valid_until': product_data.get('offers', {}).get('priceValidUntil', None),
    'item_condition': product_data.get('offers', {}).get('itemCondition', None),
    'rating_value': product_data.get('aggregateRating', {}).get('ratingValue', None),
}

# Print extracted values
print(json.dumps(extracted_values, indent=2))

{
  "name": "40oz. Stainless Steel Insulated Tumbler by Ashland\u00ae",
  "url": "https://www.michaels.com/product/40oz-stainless-steel-insulated-tumbler-by-ashland-10751061",
  "description": "Enjoy your favorite hot or cold beverage in this stainless steel tumbler. Perfect for personal use or to give as a gift, this large capacity tumbler has a convenient handle so you can take your drink on the go. Add personalized touches to your tumbler by using vinyl to make it one of a kind!Details:Available in multiple colors40 fl. oz. (1.18 L)Wash prior to first useLid is top rack dishwasher safeHand wash stainless steel vesselDo not use any abrasivesStraw or sip lidLeak proofFits in cup holdersBPA freeStainless steel, plastic and siliconeWARNING: Do not use in microwave or convection oven. Do not place in freezer. Keep out of reach of children when tumbler has hot liquid inside. Do not use straw for hot beverages.",
  "category": "Travel Drinkware",
  "color": "Yellow",
  "brand_name": "Ashla